## 80. ID番号への変換
1. ~~単語をIDに変換する辞書を作成・保存(重いため,2回目以降はセル2を実行)~~
2. 辞書の読み込み
3. 単語列をID列に変換する関数

In [22]:
# pklファイルから読み込み
import pickle

with open('../data/ch09/name_to_id.pkl', 'rb') as tf:
    name_to_id = pickle.load(tf)

In [23]:
# 与えられた単語列に対し, ID番号の列を返す関数
# ch06ではCountVectorizerを利用したため, 今回もCountVectorizerを活用する
from sklearn.feature_extraction.text import CountVectorizer

def convert_words_to_ids(words):
    '''
    input :words(単語列)
    output:ids(ID番号列)
    '''
    # analyzer: 単語列に前処理を加え, listに変換する関数
    analyzer = CountVectorizer().build_analyzer()
    word_list = analyzer(words)
    
    ids = []
    for word in word_list:
        if word in name_to_id:
            ids.append(name_to_id[word])
        else:
            ids.append(0)  # 未知語の場合, IDを0とする
    
    return ids

## prepare
1. 語彙数の取得
2. 学習データの用意(ラベル)
3. 学習データの用意(特徴量)
4. 乱数の種を固定

In [24]:
# 語彙数の取得(ID:0の単語はまとめて1語とする), 未知語, paddingを考慮
vocab_size = max(name_to_id.values())+2

In [25]:
# 訓練・検証・評価データの用意
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

# ラベル: ch08の出力を利用
Y_train = np.loadtxt('../data/ch08/Y_train.txt')
Y_valid = np.loadtxt('../data/ch08/Y_valid.txt')
Y_test = np.loadtxt('../data/ch08/Y_test.txt')

# pytorch用に変換
Y_train_long = torch.tensor(Y_train, dtype=torch.int64)
Y_valid_long = torch.tensor(Y_valid, dtype=torch.int64)
Y_test_long = torch.tensor(Y_test, dtype=torch.int64)

In [27]:
# 特徴量: convert_words_to_ids(80)を利用
def convert_text_to_features(fname):
    '''
    input :fname
    output:features(tensor)
    '''
    with open(fname, encoding='utf-8') as f:
        lines = f.readlines()
    
    # id列(list)のリストに変換
    ids_list = [convert_words_to_ids(line) for line in lines]
    
    # id列(tensor)のリストに変換
    ids_tensor = [torch.tensor(ids, dtype=torch.int64) for ids in ids_list]
    
    # 最大のid+1(vocab_size-1)でパディング
    features = torch.nn.utils.rnn.pad_sequence(ids_tensor, batch_first=True, padding_value=vocab_size-1)
    
    return features

# 特徴量抽出
X_train_long = convert_text_to_features('../data/ch06/train.txt')
X_valid_long = convert_text_to_features('../data/ch06/valid.txt')
X_test_long = convert_text_to_features('../data/ch06/test.txt')

In [28]:
# 乱数シードの固定
import random

def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

## 81. RNNによる予測
1. nn.RNNの定義
2. ~~RNNを用いた予測~~

In [96]:
# RNNの定義
class RNN1(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, batch_size, num_layers=1):
        # 層の定義
        super().__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        self.emb = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab_size-1)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True, num_layers=self.num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def init_hidden(self, batch_size=None):
        if not batch_size:
            batch_size = self.batch_size
        self.hidden_state = torch.zeros(self.num_layers, batch_size, self.hidden_size)
    
    def forward(self, x):
        x = self.emb(x)
        x_rnn, self.hidden_state =self.rnn(x, self.hidden_state)
        x = self.fc(x_rnn[:,-1,:])
        return x

## 82. 確率的勾配降下法による学習
1. accuracy計測関数の定義
2. nn.RNNを用いて学習

In [11]:
def measure_loss_accuracy1(model, criterion, dataloader):
    '''
    input : model, criterion, dataloader
    output: loss, accuracy
    '''
    running_loss = 0
    correct_num = 0
    for X, Y in dataloader:
        model.init_hidden()
        predict_y = model.forward(X)
        
        # lossの計算
        loss = criterion(predict_y, Y)
        running_loss += loss.item()
        
        # accuracyの計算
        predict_label = torch.max(predict_y, 1)[1]
        if predict_label == Y:
            correct_num += 1
    
    loss_avg = running_loss/len(dataloader)
    accuracy = correct_num/len(dataloader)
    
    return loss_avg, accuracy

In [12]:
import torch.optim as optim

# ハイパーパラメータの設定
embedding_dim = 300
hidden_size = 50
max_epoch = 10
lr = 0.002  # 評価データのaccuracyが高くなるものを選択

# dataloaderの定義
train_dataset = torch.utils.data.TensorDataset(X_train_long, Y_train_long)
train_dataloader = torch.utils.data.DataLoader(train_dataset)

valid_dataset = torch.utils.data.TensorDataset(X_valid_long, Y_valid_long)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset)

# モデルの定義
fix_seed(42)
rnn1 = RNN1(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_size=hidden_size, output_size=4, batch_size=1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(rnn1.parameters(), lr)

# SGDによる学習
for epoch in range(max_epoch):
    running_loss = 0
    for X, Y in train_dataloader:
        # forward
        rnn1.init_hidden()
        predict_y = rnn1.forward(X)
        loss = criterion(predict_y, Y)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # 更新
        optimizer.step()
        running_loss += loss.item()
    
    # loss, accuracyの表示
    rnn1.init_hidden()
    valid_loss, valid_acc = measure_loss_accuracy1(rnn1, criterion, valid_dataloader)
    train_loss, train_acc = measure_loss_accuracy1(rnn1, criterion, train_dataloader)
    train_loss = running_loss/len(train_dataloader)
    
    print(f'epoch: {epoch}')
    print(f'   train loss: {train_loss}\ttrain acc: {train_acc}')
    print(f'   valid loss: {valid_loss}\tvalid acc: {valid_acc}')

epoch: 0
   train loss: 1.162160323392314	train acc: 0.47098464994384126
   valid loss: 1.163287965658896	valid acc: 0.44236526946107785
epoch: 1
   train loss: 1.126209333208461	train acc: 0.5584986896293523
   valid loss: 1.1450665554779018	valid acc: 0.5224550898203593
epoch: 2
   train loss: 1.0570924885219535	train acc: 0.6138150505428679
   valid loss: 1.102347901102133	valid acc: 0.5681137724550899
epoch: 3
   train loss: 1.0247162953911761	train acc: 0.6557469112691876
   valid loss: 1.033805440553648	valid acc: 0.6317365269461078
epoch: 4
   train loss: 0.972744168231814	train acc: 0.6744664919505803
   valid loss: 1.008454257142758	valid acc: 0.6347305389221557
epoch: 5
   train loss: 0.9134693595248465	train acc: 0.698427555222763
   valid loss: 0.9795215310874635	valid acc: 0.6497005988023952
epoch: 6
   train loss: 0.8789332116128007	train acc: 0.7185511044552602
   valid loss: 0.9084338675515202	valid acc: 0.6848802395209581
epoch: 7
   train loss: 0.8511899708118533	trai